# ESGI(nvestment)

In [1]:
### package
import os 
import pandas as pd
import numpy as np
import copy
os.chdir("C:/Users/user/Desktop/ESG데이터/ESGI")

In [2]:
### Data

## ESG데이터
raw_esg = pd.read_excel("ESG_DATA.xlsx",converters={'종목코드':str})

## Environmental
E = pd.read_excel("E_final.xlsx",converters={'종목코드':str})

## Social
S = pd.read_excel("S_final.xlsx",converters={'종목코드':str})

## Governance
G = pd.read_excel("G_final.xlsx",converters={'종목코드':str})

## ESG
ESG = pd.read_excel("ESG_final.xlsx",converters={'종목코드':str})

## 미래에셋증권 데이터
raw_mirae = pd.read_excel("2022빅데이터페스티벌_FINANCE_info.xlsx", sheet_name = 'finance_data')

In [3]:
### 분석대상 key 변수 : 분석대상에서 SBS미디어홀딩스, 에스케이머티리얼즈 제외
raw_esg = raw_esg[-raw_esg['종목코드'].isin(['101060','036490'])]
code = raw_esg['종목코드']

### 미래에셋제공데이터
- BASE_YM : 회계년월
- ASSET_SUM : 자산총계
- CAPITAL_SUM : 자본총계
- DEBT_SUM : 부채총계
- SALE_AMT : 매출액
- BIZ_PROFIT : 영업이익
- THIS_TERM_PROFIT : 당기순이익_순손실
- FLOW_ASSET : 유동자산
- FLOW_DEBT : 유동부채
- INTEREST_COST : 이자비용
- CAPITAL_AMT : 자본금
- PROFIT_SURPLUS_AMT : 이익잉여금
- CAPITAL_SURPLUS_AMT : 자본잉여금

In [4]:
### 미래에셋데이터

## 종목코드 수정
mirae = copy.deepcopy(raw_mirae)
mirae['CODE'] = [str(i.replace("A", "")) for i in mirae['CODE']]
mirae = mirae[mirae['CODE'].isin(code)].rename(columns = {'CODE':'종목코드'})

mirae = mirae[['종목코드','BASE_YM','CO_NM','ASSET_SUM','CAPITAL_SUM','DEBT_SUM','SALE_AMT',
                   'BIZ_PROFIT','THIS_TERM_PROFIT','FLOW_ASSET','FLOW_DEBT','INTEREST_COST',
                   'CAPITAL_AMT','PROFIT_SURPLUS_AMT','CAPITAL_SURPLUS_AMT']] 
mirae['BASE_YM']

## 2020 ~ 2021개년치 데이터 추출
y = ['2020','2021']
m = ['01','02','03','04','05','06','07','08','09','10','11','12']
ym = []
for i in range(len(y)) :
    for j in range(len(m)) :
        ym.append(y[i]+m[j])

ym = list(map(int, ym))

mirae = mirae[mirae['BASE_YM'].isin(ym)]
mirae['BASE_YM'] = list(map(str,mirae['BASE_YM']))
mirae['YEAR'] = 1
mirae.loc[mirae['BASE_YM'].str.contains("2020"),"YEAR"] = "2020"
mirae.loc[mirae['BASE_YM'].str.contains("2021"),"YEAR"] = "2021"

### 투자지표생성
* 투자지표는 생성시, 손익계산서의 항목은 년 기준으로 모두 합산하며 재무제표는 가장 마지막 회계년도의 값을 선택

In [5]:
### 매출증가율, 영업이익증가율
a = mirae.groupby(["종목코드","YEAR"])[['SALE_AMT','BIZ_PROFIT','THIS_TERM_PROFIT']].sum().reset_index()
a['매출증가율'] = a.groupby('종목코드')['SALE_AMT'].apply(lambda x : (x-x.shift(1))/abs(x.shift(1)))
a['영업이익증가율'] = a.groupby('종목코드')['BIZ_PROFIT'].apply(lambda x : (x-x.shift(1))/abs(x.shift(1)))
a = a[a['YEAR'] == "2021"]
a = a.drop(["BIZ_PROFIT","SALE_AMT","YEAR"],axis=1)

In [6]:
### 유동비율, 부채비율, 자기자본비율, ROE, ROA

## 변수추출 및 종목코드 기준 2020년도, 2021년도 마지막 행 추출 (재무제표변수)
b = mirae[['종목코드','BASE_YM','ASSET_SUM','CAPITAL_SUM','DEBT_SUM','FLOW_ASSET','FLOW_DEBT','YEAR','PROFIT_SURPLUS_AMT','CAPITAL_SURPLUS_AMT','CAPITAL_AMT']].drop_duplicates("종목코드",keep='last')

## 지표생성
new_mirae = pd.merge(left = a, right = b.drop("YEAR",axis=1), how = 'left', on = '종목코드')
new_mirae['유동비율'] = new_mirae['FLOW_ASSET']/new_mirae['FLOW_DEBT']
new_mirae['부채비율'] = new_mirae['DEBT_SUM']/new_mirae['CAPITAL_SUM']
new_mirae['자기자본비율'] = (new_mirae['CAPITAL_AMT'] + new_mirae['CAPITAL_SURPLUS_AMT'] + new_mirae['PROFIT_SURPLUS_AMT'])/new_mirae['ASSET_SUM']
new_mirae['ROE'] = new_mirae['THIS_TERM_PROFIT'] / new_mirae['CAPITAL_SUM']
new_mirae['ROA'] = new_mirae['THIS_TERM_PROFIT'] / new_mirae['ASSET_SUM']

## 최종 dataframe
new_mirae = new_mirae[['종목코드','매출증가율','영업이익증가율','유동비율','부채비율','자기자본비율','ROE','ROA']]

In [7]:
### 결측값 확인
np.sum(new_mirae.isna())

종목코드        0
매출증가율       1
영업이익증가율     0
유동비율       56
부채비율        0
자기자본비율      0
ROE         0
ROA         0
dtype: int64

In [8]:
### 지표들 요약통계량
final_mirae = pd.merge(left = raw_esg[['종목코드','SICS_Sector_Kr']], right = new_mirae, how = 'left', on ='종목코드')
final_mirae.describe()

,매출증가율,영업이익증가율,유동비율,부채비율,자기자본비율,ROE,ROA
count,942.000000,943.000000,887.000000,943.000000,943.000000,943.000000,943.000000
mean,0.506217,1.488912,3.166540,1.240738,0.622139,0.027042,0.027848
std,9.199526,17.348841,9.161049,2.218076,0.287793,0.263514,0.100061
min,-0.996598,-248.207110,0.001399,0.001615,-0.132549,-3.350413,-0.665487
25%,0.000102,-0.255687,0.994309,0.280394,0.454728,0.010784,0.005425
50%,0.119702,0.175219,1.520258,0.630937,0.627057,0.055379,0.031233
75%,0.274161,0.958334,2.725753,1.185008,0.795587,0.109602,0.064819
max,281.393809,300.279473,155.228791,22.823453,2.506751,0.970453,0.961470


In [9]:
## 종목 "메드팩토" 매출 기입 x -> 결측값은 영업이익 증가율로 대체
final_mirae[final_mirae['매출증가율'].isna()]
final_mirae.loc[final_mirae['매출증가율'].isna(),'매출증가율'] = -0.056894
final_mirae.groupby("SICS_Sector_Kr")['유동비율'].min()['인프라']

## 금융이 아닌 유동비율은 업종별 최소값으로 대체
impu = final_mirae.groupby("SICS_Sector_Kr")['유동비율'].min()['인프라']
final_mirae.loc[(final_mirae['SICS_Sector_Kr'] != "금융")&(final_mirae['유동비율'].isna()),"유동비율"] = impu

In [10]:
### 지표들 상하한 조정
final_mirae.loc[final_mirae['매출증가율'] >= 1,'매출증가율'] = 1
final_mirae.loc[final_mirae['매출증가율'] <= -1,'매출증가율'] = -1
final_mirae.loc[final_mirae['영업이익증가율'] >= 1,'영업이익증가율'] = 1
final_mirae.loc[final_mirae['영업이익증가율'] <= -1,'영업이익증가율'] = -1
final_mirae.loc[final_mirae['유동비율'] >= 2,'유동비율'] = 2
final_mirae.loc[final_mirae['부채비율'] >= 2,'부채비율'] = 2
final_mirae.loc[final_mirae['자기자본비율'] >= 0.5,'자기자본비율'] = 0.5
final_mirae.loc[final_mirae['자기자본비율'] <= -0.5,'자기자본비율'] = -0.5
final_mirae.loc[final_mirae['ROE'] >= 0.5,'ROE'] = 0.5
final_mirae.loc[final_mirae['ROE'] <= -0.5,'ROE'] = -0.5
final_mirae.loc[final_mirae['ROA'] >= 0.25,'ROA'] = 0.25
final_mirae.loc[final_mirae['ROA'] <= -0.25,'ROA'] = -0.25

In [11]:
### 조정 후, 지표에 대한 scoring
final_mirae['매출증가율'] = (final_mirae['매출증가율'] + 1)/2 * 100
final_mirae['영업이익증가율'] = (final_mirae['영업이익증가율'] + 1)/2 * 100
final_mirae['유동비율'] = final_mirae['유동비율']/2 * 100
final_mirae['부채비율'] = (1 - final_mirae['부채비율']/2) * 100
final_mirae['자기자본비율'] = (final_mirae['자기자본비율']/0.5 + 1)/2 * 100
final_mirae['ROE'] = (final_mirae['ROE']/0.5 + 1)/2 * 100
final_mirae['ROA'] = (final_mirae['ROA']/0.25 + 1)/2 * 100

In [12]:
### 각 지표의 합
final_mirae['성장성'] = (final_mirae['매출증가율'] + final_mirae['영업이익증가율'])/2
final_mirae.loc[final_mirae['SICS_Sector_Kr'] == "금융",'안정성'] = (final_mirae['부채비율'] + final_mirae['자기자본비율'])/2
final_mirae.loc[final_mirae['SICS_Sector_Kr'] != "금융",'안정성'] = (final_mirae['유동비율'] + final_mirae['부채비율'] + final_mirae['자기자본비율'])/3
final_mirae['수익성'] = (final_mirae['ROE'] + final_mirae['ROA'])/2

### ESGI함수
* 성장성, 안정성, 수익성 3가지의 지표의 투자자가 원하는 가중치를 둠으로, 원하는 가치에 맞는 가중치를 올려 ESG기반의 투자지표를 확인할 수 있음

In [13]:
## function of ESGI
def ESGI(growth_weight, stable_weight, profit_weight):
    esgi = pd.merge(left = ESG[['종목코드','SICS_Sector_Kr','ESG_risk']], right = final_mirae[['종목코드','성장성','안정성','수익성']], how = 'left', on = '종목코드')
    esgi['ESGI'] = (growth_weight*esgi['성장성'] + stable_weight*esgi['안정성'] + profit_weight*esgi['수익성'])*esgi['ESG_risk']
    return esgi[['종목코드','SICS_Sector_Kr','ESGI']]

# Example : 성장성 70%, 안정성 10%, 수익성 20%
ESGI(0.7,0.1,0.2)

,종목코드,SICS_Sector_Kr,ESGI
0,105560,금융,53.125264
1,316140,금융,37.800499
2,034730,기술 및 통신,28.943736
3,029780,금융,47.864419
4,055550,금융,49.673840
...,...,...,...
938,298040,자원 변환,32.744372
939,298050,자원 변환,51.094160
940,298020,소비재,44.802244
941,298000,자원 변환,33.551483


In [14]:
## Tableau시각화 데이터 추출
esgi = pd.merge(left = ESG[['종목코드','SICS_Sector_Kr','ESG_risk']], right = final_mirae[['종목코드','성장성','안정성','수익성']], how = 'left', on = '종목코드')
esgi.to_excel("C:/Users/user/Desktop/ESG데이터/Tableau/ESGI_DATA.xlsx", index = None)